In [1]:
from bs4 import BeautifulSoup
import requests
import nltk
import math
nltk.download('stopwords')
import pandas as pd
import pickle
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import networkx as nx
import networkx.algorithms.community as nx_community
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
import networkx.algorithms.community as nx_comm
import netwulf
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


# Webscraping 
Retrieving all character names from wikisite: 
"https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters"

In [70]:
# Characterlist from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_Harry_Potter_characters'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# List for characters
character_list = []
content_div = soup.find('div', {'id': 'bodyContent'})
for heading in content_div.find_all('h3'):
    # find the <ul> tag immediately following this heading
    character_ul = heading.find_next_sibling('ul')
    for character_li in character_ul.find_all('li'):
        # extract the character name from the <a> tag within the <li>, if it exists
        character_a = character_li.find('a')
        if character_a:
            character_name = character_a.text
            character_list.append(character_name)

print(character_list)

['Hannah Abbott', 'Ludo Bagman', 'Bathilda Bagshot', 'Katie Bell', 'Cuthbert Binns', 'Phineas Nigellus Black', 'Sirius Black', 'Amelia Bones', 'Susan Bones', 'Terry Boot', 'Lavender Brown', 'Slytherin', 'Charity Burbage', 'Frank Bryce', 'Alecto Carrow', 'Amycus Carrow', 'Reginald Cattermole', 'Cho Chang', 'Percy Weasley', 'Crabbe', 'Vincent Crabbe', 'Colin Creevey', 'Dennis Creevey', 'Dirk Cresswell', 'Barty Crouch Sr', 'Barty Crouch Jr', 'John Dawlish', 'Fleur Delacour', 'Dedalus Diggle', 'Amos Diggory', 'Cedric Diggory', 'Elphias Doge', 'Antonin Dolohov', 'Aberforth Dumbledore', 'Albus Dumbledore', 'Ariana Dumbledore', 'Dudley Dursley', 'Marge Dursley', 'Petunia Dursley', 'Vernon Dursley', 'Marietta Edgecombe', 'Arabella Figg', 'Argus Filch', 'Justin Finch-Fletchley', 'Seamus Finnigan', 'Mundungus Fletcher', 'Filius Flitwick', 'Cornelius Fudge', 'Marvolo Gaunt', 'Merope Gaunt', 'Morfin Gaunt', 'Anthony Goldstein', 'Goyle', 'Gregory Goyle', 'Hermione Granger', 'Draco Malfoy', 'Eastern

In [71]:
# Ikke optimal løsning, men fikser alle fejlene, så vi kan smide en liste til API'en lidt hurtigt. 
replacements = {
    "Slytherin": "Millicent Bulstrode",
    "Percy Weasley": "Penelope Clearwater",
    "Draco Malfoy": "Astoria Greengrass",
    "Eastern European": "Gregorovitch",
    "Diagon Alley": "Madam Malkin",
    "Horace Slughorn": "Cormac McLaggen",
    "Vanishing Cabinet": "Graham Montague",
    "Harry Potter and the Cursed Child": "Delphi Riddle",
    "Merope Gaunt": "Tom Riddle Sr.",
    "Lord Voldemort": "Thomas Marvolo Riddle",
    "Bogrod": "Gringotts",
    "Gringotts": "Gornuk",
    "Black family": "Kreacher",
    "Death Eater" : "Travers",
    "Barty Crouch Sr": "Winky"}

for i in range(len(character_list)):
    if character_list[i] in replacements:
        character_list[i] = replacements[character_list[i]]

additions = ["Gabrielle Delacour", 
             "Kendra Dumbledore", 
             "Percival Dumbledore", 
             "Nicolas Flamel", 
             "Griselda Marchbanks", 
             "Eloise Midgen", 
             "Theodore Nott", 
             "Nott Sr", 
             "Albus Severus Potter", 
             "James Sirius Potter", 
             "Lily Luna Potter", 
             "Mary Riddle", 
             "Thomas Riddle", 
             "Demelza Robins", 
             "Salazar Slytherin", 
             "Romilda Vane", 
             "Hugo Weasley", 
             "Rose Weasley",
             "Millicent Bulstrode",
             "Blaise Zabini",
             "Sir Cadogan",
             "Scabior",
             "Trevor"]

print(character_list)

['Hannah Abbott', 'Ludo Bagman', 'Bathilda Bagshot', 'Katie Bell', 'Cuthbert Binns', 'Phineas Nigellus Black', 'Sirius Black', 'Amelia Bones', 'Susan Bones', 'Terry Boot', 'Lavender Brown', 'Millicent Bulstrode', 'Charity Burbage', 'Frank Bryce', 'Alecto Carrow', 'Amycus Carrow', 'Reginald Cattermole', 'Cho Chang', 'Penelope Clearwater', 'Crabbe', 'Vincent Crabbe', 'Colin Creevey', 'Dennis Creevey', 'Dirk Cresswell', 'Winky', 'Barty Crouch Jr', 'John Dawlish', 'Fleur Delacour', 'Dedalus Diggle', 'Amos Diggory', 'Cedric Diggory', 'Elphias Doge', 'Antonin Dolohov', 'Aberforth Dumbledore', 'Albus Dumbledore', 'Ariana Dumbledore', 'Dudley Dursley', 'Marge Dursley', 'Petunia Dursley', 'Vernon Dursley', 'Marietta Edgecombe', 'Arabella Figg', 'Argus Filch', 'Justin Finch-Fletchley', 'Seamus Finnigan', 'Mundungus Fletcher', 'Filius Flitwick', 'Cornelius Fudge', 'Marvolo Gaunt', 'Tom Riddle Sr.', 'Morfin Gaunt', 'Anthony Goldstein', 'Goyle', 'Gregory Goyle', 'Hermione Granger', 'Astoria Greengr

In [73]:
# API for facts about Harry Potter characters
API = {"data":{"message":"Welcome to the Potter DB API!","api_routes":["/v1/books/:book_id/chapters","/v1/books/:book_id/chapters/:id","/v1/books","/v1/books/:id","/v1/characters","/v1/characters/:id","/v1/movies","/v1/movies/:id","/v1/potions","/v1/potions/:id","/v1/spells","/v1/spells/:id"]}}
API

{'data': {'message': 'Welcome to the Potter DB API!',
  'api_routes': ['/v1/books/:book_id/chapters',
   '/v1/books/:book_id/chapters/:id',
   '/v1/books',
   '/v1/books/:id',
   '/v1/characters',
   '/v1/characters/:id',
   '/v1/movies',
   '/v1/movies/:id',
   '/v1/potions',
   '/v1/potions/:id',
   '/v1/spells',
   '/v1/spells/:id']}}

In [79]:
# API link
BASE_URL = "https://api.potterdb.com/"
VERSION = "v1/"
RESOURCE = "characters"

my_url = BASE_URL + VERSION + RESOURCE
print(my_url)

https://api.potterdb.com/v1/characters


In [129]:
# Loading data
params = {'query': '{}'.format(character_list), "offset":0,
           "limit":1}
r = requests.get(my_url, params=params)
data = r.json()["data"]

# Setting all names from characterlist as keys in dictionary holding attributes
all_data = {}
for i in range(len(character_list)):
    all_data[character_list[i]] = None

# Comparing names from data with characterlist
for i in data:
    name = (i['attributes']['name'])
    for j in all_data.keys():
        #print(j)
        if name == j:
            print('yes')
            print(name)
            all_data[j] = i['attributes']['blood_status']

#print(all_data['blood_status'])

yes
Aberforth Dumbledore
yes
Albert Runcorn
yes
Alecto Carrow


In [186]:
# Using api and making dataframe of data and attributes without wiki site
# Loading data
params = {'query': '{}'.format(character_list), "offset":0,
           "limit":1}
r = requests.get(my_url, params=params)
data = r.json()["data"]

names_from_data = []
for i in data:
    names_from_data.append(i['attributes']['name'])

print(data[0]['attributes'])

all_data = {}
for i in range(len(names_from_data)):
    all_data[names_from_data[i]] = {}

for i in data:
    name = (i['attributes']['name'])
    for j in all_data.keys():
        if name == j:
            all_data[j]['Blood type'] = i['attributes']['blood_status']
            all_data[j]['House'] = i['attributes']['house']
            all_data[j]['Death time'] = i['attributes']['died']
            all_data[j]['Alias'] = i['attributes']['alias_names']

{'slug': '1992-gryffindor-vs-slytherin-quidditch-match-spectators', 'name': '1992 Gryffindor vs Slytherin Quidditch match spectators', 'born': None, 'died': None, 'gender': None, 'species': None, 'height': None, 'weight': None, 'hair_color': None, 'eye_color': None, 'skin_color': None, 'blood_status': None, 'marital_status': None, 'nationality': None, 'animagus': None, 'boggart': None, 'house': None, 'patronus': None, 'alias_names': None, 'family_members': None, 'jobs': None, 'romances': None, 'titles': None, 'wands': None, 'image': None, 'wiki': 'https://harrypotter.fandom.com/wiki/1992_Gryffindor_vs_Slytherin_Quidditch_match_spectators'}


In [182]:
# Data as dataframe
df_data = pd.DataFrame.from_dict(all_data, orient='index')
#df_data = df_data.dropna(subset=['Blood type'])
df_data

,Blood type,House,Death time,Alias
1992 Gryffindor vs Slytherin Quidditch match spectators,None,None,None,None
1996 Gryffindor Quidditch Keeper trials spectators,None,None,None,None
19th-century Flying Instructor,None,None,None,None
19th-century Herbology professor,None,None,None,None
19th-century Potions professor,None,None,None,None
...,...,...,...,...
Alexia Walkin Black,Pure-blood,Slytherin (possibly),Before 1996,None
Alexis Marie,None,Gryffindor,None,None
Alex Sykes,None,Slytherin,None,None
Alex Wednesbury,None,Gryffindor,None,None
